# **KSP 5.0 Selection Assignment**





In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from matplotlib.colors import ListedColormap
import math
from scipy.stats import norm

# Section 1: Calculating the Redshift of Galaxies

You are given the data for all the galaxies in the cluster in the file 'galaxy.csv'.

This includes the name of the galaxy, observed wavelength of the $H\alpha$ line $\lambda_{H\alpha}$, Right Ascension at J2000.0 and Declination and J2000.0 (You can read more about RA and Dec [here](https://en.wikipedia.org/wiki/Equatorial_coordinate_system))

In [ ]:
galaxy_data = pd.read_csv('data/galaxy.csv')


Since the galaxies would have some radial velocities with respect to us, the wavelengths we observed would be [redshifted](https://en.wikipedia.org/wiki/Redshift) due to doppler effect.

Using the value of the Observed Wavelength of $H\alpha$ line (Rest Wavelength $\lambda_{H\alpha0} = 6562.8nm)$, calculate the redshift of the galaxies in our cluster.

Use this to find their radial velocities of the galaxies and plot it.

(Assume that the observed wavelength has been correscted for the motion of earth around sun, and motion of sun around the center of Milky Way. Put more simply, these are the wavelengths you would have observed if you were at rest at the center of Milky Way)

#### Calculating Redshift
Observed wavelength due to doppler effect
$$\lambda_{observed}= (1+\frac{V_{source}}{c}).\lambda_{emitted}$$
$$\frac{V_{source}}{c}=\frac{\lambda_{observed}-\lambda_{emitted}}{\lambda_{emitted}}$$

In [ ]:
normal_wavelength = 6562.8#wavelenght of Halphaline
galaxy_data['Redshift'] = (galaxy_data['Observed Wavelength of H-alpha'] - normal_wavelength) / normal_wavelength
galaxy_data['Radial Velocity (km/s)'] = galaxy_data['Redshift'] * 3e5 

Using the data from galaxy.csv, answer the following questions.

For your convenience, the RA and Dec data has been converted from the format $a^\circ b^m c^s$ to $xx^\circ$ below


### Calculating angular Size of galaxy cluster
#### Assumptions:
- RA and Dec of centre of Galaxy cluster is given by mean of RA`s and mean of Dec's of its corresponding galaxies 
- around 95% of galaxie`s angular sepration(from center) lie in the range of 0 to maximum_angular_sep-0.1, angular size of cluster is taken as 2*(max_angularsep-0.1)  

#### Calculating angular seperation:

Distance $d$ between two points on a sphere with latitudes(dec in celestial sphere) $\omega_{1}$ and $\omega_{2}$ and longitudes(RA in celestial sphere) $\delta_{1}$ and $\delta_{2}$ is given by Haversine distance formula
$$ d = 2r\arcsin({\sqrt{{\sin^2{(\frac{\omega_{1}-\omega_{2}}{2})}} + \cos{\omega_{1}}.\cos{\omega_{2}}.{\sin^2{(\frac{\delta_{1}-\delta_{2}}{2})}}}})$$
Where the angular seperation is $\frac{d}{r}$,which is 
$$\theta =2\arcsin({\sqrt{{\sin^2{(\frac{\omega_{1}-\omega_{2}}{2})}} + \cos{\omega_{1}}.\cos{\omega_{2}}.{\sin^2{(\frac{\delta_{1}-\delta_{2}}{2})}}}})$$
you can know more about haversine distance [here](https://en.wikipedia.org/wiki/Haversine_formula)

In [ ]:
temp = galaxy_data['RA J2000.0'].str.split(' ', expand=True)
galaxy_data['RA J2000.0'] = temp[0].astype(float) + temp[1].astype(float) / 60 + temp[2].astype(float) / 3600
temp = galaxy_data['Dec J2000.0'].str.split(' ', expand=True)
galaxy_data['Dec J2000.0'] = temp[0].astype(float) + temp[1].astype(float) / 60 + temp[2].astype(float) / 3600
mean_ra=galaxy_data['RA J2000.0'].mean()
mean_dec=galaxy_data['Dec J2000.0'].mean()
def haversine_distance(ra1, dec1, ra2, dec2):
    ra1, dec1, ra2, dec2 = map(math.radians, [ra1, dec1, ra2, dec2])
    d_ra = ra2 - ra1
    d_dec = dec2 - dec1
    a = math.sin(d_dec/2)**2 + math.cos(dec1) * math.cos(dec2) * math.sin(d_ra/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # in radians
    return c
angular_sep=[]
for i in range(0,len(galaxy_data['Dec J2000.0'])):
    angular_sep.append(haversine_distance(mean_ra,mean_dec,galaxy_data['RA J2000.0'][i],galaxy_data['Dec J2000.0'][i]))
angular_sep1=np.array(angular_sep)
max_angularsep=max(angular_sep)
print("Total galxies",np.sum(angular_sep1<max_angularsep))
print("ang_sep<max-1",np.sum(angular_sep1<max_angularsep-0.1))
print("angsep<max-0.15",np.sum(angular_sep1<max_angularsep-0.15))
print(np.sum(angular_sep1<max_angularsep-0.17))
angular_size_of_cluster=2*(max_angularsep-0.1)
print("Approximate angular size of cluster is" ,end=" ")
print(angular_size_of_cluster)


***

###Q1.1. What is the approximate angular size of the galaxy cluster?

About  0.4509281425787243 radians


***

###Q1.2. Can you comment anything about the rotation of the galaxies around the center of the cluster.
HINT: You may find it useful to plot a scatter plot of RA and DEC with radial velocity as colormap.

From the below graph we can see that the galaxies which had velocity above average are dominating over those having less in region of dec>avg dec and the no. of galaxies which had velocity below average are dominating over those which had above in region of dec< avg dec.So, there can be Rotation of galaxies around center of cluster in the orientation in such a way that, it upper part (dec > avg dec) has a velocity component along the direction of radial velocity of the cluster, and its lower part in the opposite direction to radial velocity.We can see the shift in color of points(representing change in speed) with increase of declination.
***

In [ ]:
average_rad=galaxy_data['Radial Velocity (km/s)'].mean()
above_avg_high_dec = galaxy_data[(galaxy_data['Radial Velocity (km/s)'] > average_rad) & (galaxy_data['Dec J2000.0'] > mean_dec)]
above_avg_low_dec = galaxy_data[(galaxy_data['Radial Velocity (km/s)'] > average_rad) & (galaxy_data['Dec J2000.0'] <= mean_dec)]
below_avg_high_dec = galaxy_data[(galaxy_data['Radial Velocity (km/s)'] < average_rad) & (galaxy_data['Dec J2000.0'] > mean_dec)]
below_avg_low_dec = galaxy_data[(galaxy_data['Radial Velocity (km/s)'] < average_rad) & (galaxy_data['Dec J2000.0'] <= mean_dec)]
cmap = ListedColormap(['blue', 'green', 'red', 'purple'])
plt.figure(figsize=(17, 6))
plt.subplot(1,2,1)
plt.title('Radial Velocities of Galaxies')
scatter=plt.scatter(galaxy_data['RA J2000.0'],galaxy_data["Dec J2000.0"],c=galaxy_data['Radial Velocity (km/s)'],cmap='viridis')
plt.xlabel('RA (hours)')
plt.ylabel('Dec (degrees)')
plt.colorbar(scatter, label='Radial Velocity (km/s)')
plt.subplot(1,2,2)
plt.title('Radial Velocities of Galaxies')
plt.scatter(below_avg_high_dec['RA J2000.0'], below_avg_high_dec['Dec J2000.0'], c='orange', label='Radial Velocity < avg km/s & Dec > avg')
plt.scatter(above_avg_high_dec['RA J2000.0'], above_avg_high_dec['Dec J2000.0'], c='brown', label='Radial Velocity > avg km/s & Dec > avg')
plt.scatter(above_avg_low_dec['RA J2000.0'], above_avg_low_dec['Dec J2000.0'], c='blue', label='Radial Velocity > avg km/s & Dec <= avg')
plt.scatter(below_avg_low_dec['RA J2000.0'], below_avg_low_dec['Dec J2000.0'], c='lightgreen', label='Radial Velocity < avg km/s & Dec <= avg')
plt.legend()
plt.xlabel('RA (hours)')
plt.ylabel('Dec (degrees)')
print("velocity and declination","   ","no. of galaxies")
print("v>avg_vel and dec>avg_dec","   ",len((above_avg_high_dec['RA J2000.0'])))
print("v>avg_vel and dec<avg_dec","   ",len((above_avg_low_dec['RA J2000.0'])))
print("v<avg_vel and dec>avg_dec","   ",len((below_avg_high_dec['RA J2000.0'])))
print("v<avg_vel and dec<avg_dec","   ",len((below_avg_low_dec['RA J2000.0'])))
plt.show()

#Section 2: Calibrating Cepheids Data

NOTE: Although there are no prerequisities for this section, it relies a bit on knowledge about cepheids and their Period-Luminosity Relation. Necessary resource links have been provided wherever needed. You are strongly encouraged to use the internet to aid as a reference whenever needed.
***

The [Gaia Mission](https://en.wikipedia.org/wiki/Gaia_(spacecraft) aims at cataloging and create a precise three-dimensional map of astronomical objects throughout the Mily Way. It maps the motion of the stars, which encode the origin and subsequent evolution of the Milky Way. The spectrophotometric measurements provide detailed physical properties of all stars observed, characterizing their luminosity, effective temperature, gravity and elemental composition.





In [ ]:
gaia_data = pd.read_csv('data/gaia.csv')

Cepheids are a particular type of variable stars that pulsates radially, varying in both diameter and temperature. They **changes in brightness**, with a **well-defined stable period and amplitude**.

The file 'gaia.csv' includes data for about 5000 Cepheid Variable Stars in our own galaxy. In particular, the file contains the following data fields:
   
*   Unique ID of the Source Star
*   Parallax (in milli arcseconds) : You can read more about parallax [here](https://en.wikipedia.org/wiki/Stellar_parallax)
*   Mean **Apparent** Magnitude ($m$)
*   TIme period of Pulsations P(in days)



Classical Cepheids follow the [Period Luminosity Law](https://en.wikipedia.org/wiki/Period-luminosity_relation#The_relations), also commonly known as the Leavitt Law.

It basically states that there is an empirical relation between the pulsation period $(P)$ of a Cepheid Star and it's **Absolute Magnitude** ($M$) of the form -

#### $\begin{equation}M = \alpha \log_{10}(P \text{(in days)}) + \beta\end{equation}$
***

### Your aim is to verify Leavitt Law for cepheids using the given data, and find the values of $\alpha$ and $\beta$.

You may find it helpful to use the function 'poly_fit' in numpy for this.

#### Finding distance of cepheid:
$$\tan{p}= \frac{1 au}{d} \qquad \text{(where $p$ = parallax and d is distance in au(astronomical units) )}$$
since we deal with very small angles,using single angle approximation $\tan{x}\approx x$ ,which implies
$$d=\frac{1 au}{p}$$
d is obtained in parsecs,on taking p in arcseconds.{ 1parsec is the distance of a star whoes parallax is 1 radian}
#### Absolute and Apparent Magnitude
Relation between M(absolute magnitude) and m(Apparent Magnitude) is given as 
$$
M=m-5(\log_{10}d_{pc}-1) \qquad  \text{ where d is distance in parsec}
$$

After Fitting the data using polyfit ,the coefficients according to leavitt theorum(1)  are obtained as,

$$\alpha= -1.7501625568866086  , \beta= 0.4529616510303397
$$
We can see the from the graph, that data is consistent with Leavitts Law (1),offcourse with a bit a error margin.

In [ ]:
#converting them into numeric data
gaia_data['mean apparent magnitude']=pd.to_numeric(gaia_data['mean apparent magnitude'],errors='coerce')
gaia_data['time period']=pd.to_numeric(gaia_data['time period'],errors='coerce')
gaia_data['parallax']=pd.to_numeric(gaia_data['parallax'],errors='coerce')
gaia_data['distance']=1/((gaia_data['parallax'])/1000)
#absolute magnitude=apparent-5(log(d)-1) (d is in parsec)
gaia_data['absolute_mag']=gaia_data['mean apparent magnitude']-5*(np.log10(gaia_data['distance'])-1)
gaia_data['x-cord']=np.log10(gaia_data['time period'])
coeff=np.polyfit(gaia_data['x-cord'],gaia_data['absolute_mag'],1)
alpha=coeff[0]
beta=coeff[1]
print("alpha   ",alpha)
print("beta   ",beta)
plt.scatter(gaia_data['x-cord'],gaia_data['absolute_mag'],label="data")
plt.plot(gaia_data['x-cord'], alpha*gaia_data['x-cord'] + beta, color='red', label="best Fit line")
plt.legend()
plt.show()

***
### Some useful information about the Magnitude system

The brightness of a star as seen by some observer is dependent on the observer's distance from the star as well as the intrinsic brightness of the star (ignoring any absorption in the intervening space).

In astronomy, brightness of a star, as seen by some observer, is measured in terms of its **apparent magnitude ($m$)**. For two stars (1 and 2) with fluxes $f_1$ and $f_2$ respectively, their apparent magnitudes $m_1$ and $m_2$ are related by:

$m_1 - m_2 = 2.5 \log_{10} \left( \frac{f_1}{f_2} \right)$

**Absolute magnitude ($M$)** of any star is its apparent magnitude if the star was exactly 10 parsecs away from the observer. Thus, $M$ only depends on the intrinsic brightness of the star.

***


# Section 3: Getting the distances to our galaxies

Now that we have our period-luminosity relation, we can observe cepheids in the galaxies in our cluster and use this to get the distances to our galaxies. (Yes, this is possible! Hubble Telescope actually observed Cepheids in the galaxies in Virgo Cluster)


For this, you have been given the data file 'cepheid.csv' which contains the following data fields:
*   Unique ID of the Cepheid
*   Name of the Galaxy to which it belongs
*   Mean Apparent Magnitude (m)
*   Time Period of Pulsation (in days)

In [ ]:
#your code here
cepheid_data = pd.read_csv('data/cepheid.csv')

Use the Period-Luminosity Law derived in Section-2 and the above data to find an estimate for the distance to each galaxy in the cluster.

Make sure to handle outliers in the above data. The data may contain some cepheids that actually DO NOT belong to the given galaxy, but it just seems to us that this is the case since they fall along the same line of sight.
***

#### Finding Absolute Magnitude using Leavitts theorum

In [ ]:
cepheid_data['Period']=pd.to_numeric(cepheid_data['Period'],errors='coerce')
cepheid_data['Apparent Magnitude']=pd.to_numeric(cepheid_data['Apparent Magnitude'],errors='coerce')
cepheid_data['absolute']=alpha*np.log10(cepheid_data['Period'])+beta

# Section 4: Getting the Hubble's Constant

The famous [Hubble's Law](https://en.wikipedia.org/wiki/Hubble%27s_law) is the observation that galaxies are moving away from Earth at speeds proportional to their distance.

$v = H_0 d$

where $H_0$ is the proportionalaity constant, also known as the Hubble's Constant.

Now that we have an estimate for the radial velocities and distance to the galaxies in our cluster, we need to find an estimate of the Hubble Constant (H).

Fit gaussians on your velocity and distance data and report the mean and standard deviation of the same. Also plot the histograms and the fitted gaussian.

Finding distance using  Equation :  $M=m-5(\log_{10}d_{pc}-1)$

In [ ]:

cepheid_data['distance']=np.power(10,(cepheid_data['Apparent Magnitude']-cepheid_data['absolute'])/5+1)

Assuming that all the galaxies given are the part of a galaxy cluster,And the distance of the galaxies from us are of the same order ,i.e relatively near to each other.In the given Data there are some galaxies whoes distance is anamlous  with order of $10^3-10^4$ whereas the most of others had a order around $10^7$.Those cephieds may not be a part of galaxy of the cluster under consideration.They seem us to as if they were,since they fall in same the line of sight.

In [ ]:

cepheids_with_anamolous_dist=np.sum(cepheid_data['distance']/10000000 <0.001) # about 475 #the number doesnt change from 1 to 0.001 , that too about chanage of 3
cepheids_with_relatable_dist=np.sum((cepheid_data['distance']/10000000 >1) & (cepheid_data['distance']/10000000 < 2))#about 4494#the number didnt change much from (1 to 1.4) front limit
print("cepheids with anamolous_dist: ",cepheids_with_anamolous_dist)
print("cepheids with relatable dist: ",cepheids_with_relatable_dist)
data_of_galaxy_dist=cepheid_data[(cepheid_data['distance']/10000000 >1) & (cepheid_data['distance']/10000000 < 2)]
#taking average of distance of cepheids present in the same galaxy 
average_galaxy_dist=data_of_galaxy_dist.groupby('Galaxy_ID')['distance'].mean()

#### creating a new_data constituting galaxyname,avg_distance,Apparent Mag,Radial velcoity(km/s)
 - Data of Absolute Mag of the galaxy, is added , which is obtained using the relation between apparent Mag and absolute mag.
 - Absolute mag is usefull for calulating weight of the galaxy(due to stars).

In [ ]:
galaxyname_avg_dist=pd.DataFrame({'Name':average_galaxy_dist.index ,'avg_dist':average_galaxy_dist.values})
new_data_vel=galaxy_data[['Name','Radial Velocity (km/s)','Apparent Mag']]
new_data=pd.merge(galaxyname_avg_dist,new_data_vel,on='Name',how='inner')
#assuming distance of galaxy is same as distance of cepheid.
new_data['abs_MAG']=new_data['Apparent Mag']-5*(np.log10(new_data['avg_dist'])-1)

#### Finding Hubble constant
$$H=\frac{V_{mean}}{d_{mean}} \qquad  (\text{where $v_{mean}$ and $d_{mean}$ are obtained by fitting data into gaussian distribution})$$

In [ ]:

#new data contains galaxyName , galaxy distance ,galaxy velocity
mean_vel,std_vel=norm.fit(new_data['Radial Velocity (km/s)'])
mean_dist,std_dist=norm.fit(new_data['avg_dist'])
plt.figure
plt.subplot(1, 2, 1)
plt.hist(new_data['Radial Velocity (km/s)'], bins=100, density=True, alpha=0.6, color='g')
xmin, xmax = plt.xlim()
x_velocity = np.linspace(xmin, xmax, 100)
p_velocity = norm.pdf(x_velocity, mean_vel, std_vel)
plt.plot(x_velocity, p_velocity, 'k', linewidth=0.75)
plt.title("Velocity Histogram")
plt.xlabel("Velocity")
plt.ylabel("Density")
plt.subplot(1, 2, 2)
plt.hist(new_data['avg_dist'], bins=100, density=True, alpha=0.6, color='b')
xmin, xmax = plt.xlim()
x_dist = np.linspace(xmin, xmax, 100)
p_dist = norm.pdf(x_dist, mean_dist, std_dist)
plt.plot(x_dist, p_dist, 'k', linewidth=0.75)
plt.title("Distance Histogram")
plt.xlabel("Distance")
plt.ylabel("Density of v region")
plt.show()
Hubble_constant=mean_vel/mean_dist*1000000
print("Hubble constant:  ",Hubble_constant)

Using the values obtained in the previous part, report your value of $H$ along with the error margins.

#### Finding Error Margin Of Hubble constant
$$ \big(\frac{\delta H}{H}\big)^2 =  \big(\frac{\delta V}{V_{mean}}\big)^2+ \big(\frac{\delta d}{d_{mean}}\big)^2$$
where $\delta d$ ,$\delta V$ are the standard deviations of distance and velocity respectively.

In [ ]:
#error of hubble constant
sum=np.power(std_dist/mean_dist,2)+np.power(std_vel/mean_vel,2)
Error_in_hubbleconst=Hubble_constant*np.power(sum,0.5)
print("Error in Hubble constant: ",Error_in_hubbleconst)
print()
print("HUBBLE CONSTANT:  ",Hubble_constant,"+/-",Error_in_hubbleconst)

***

#### Q4.1 Compare the value of $H$ that you got from the standard $H_0$ commonly accepted today. The estimate that you got - is it the same (within error margins), higher or lower than the value of $H_0$.

Error margin: 40.3795233834894


Hubble constant: 52.48154485661434 +/- 40.3795233834894

***

#### Q4.2 What might be the reason behind this difference in values

The relation between velocity and distance ($v=H_0d$), is valid for the velocity which are due to the expansion of 
universe but the velocity of galaxies can be effected by rotation around its centre of its cluster group,and it also valid only for the positive redshift, since the velocity caused by expansion of universe is offcourse, should be away from us,i.e positive redshift.In the data there are velocities which are having  blue shift(may be due to collisons or rotation) which effects the value of hubble constant.

***

# Section 5 : Dark Matter

Dark matter is a component of the universe whose presence is discerned from its gravitational attraction rather than its luminosity.

***

#### Q5.1 ESTIMATE the total mass of the Galaxy Cluster. This includes mass due to visible matter as well as dark matter.

HINT: The [Virial Theorem](https://phys.libretexts.org/Bookshelves/Classical_Mechanics/Variational_Principles_in_Classical_Mechanics_%28Cline%29/02%3A_Review_of_Newtonian_Mechanics/2.11%3A_Virial_Theorem) provides a general equation that relates the average over time of the total kinetic energy of a stable system of discrete particles, bound by a conservative force
 $$ \left<KE\right> = -\frac{1}{2} \left<PE\right> $$



#### Relation between Velocity dispersion and Mass of Galaxy
Assuming the galaxies are having a equal mass of $m$
$$KE=\frac{1}{2}\Sigma m{v_{i}}^2=\frac{1}{2}mn\frac{\Sigma {v_i}^2}{n}=\frac{1}{2}nm{v_{rms}}^2$$
$$PE=-k\frac{G {(nm)}^2}{r}$$
Using viral theorum,
$$nm{v_{rms}}^2=k\frac{G {(nm)}^2}{r} \implies M=nm=k\frac{<V^2>r}{G}$$
you can see more about the equation [here](https://phys.libretexts.org/Bookshelves/Astronomy__Cosmology/Big_Ideas_in_Cosmology_(Coble_et_al.)/08%3A_Dark_Matter/8.04%3A_Velocity_and_Mass_Distributions_in_Galaxies)

Value of k depends on distribution of mass, Assuming the cluster to be uniform sphere of radius r, and Mass M, then 

$U=-\frac{5}{3}\frac{GM^2}{r}$ and By equipartition $<V^2>=3<{V_{rad}}^2>$ ,by considerable approximation 
$$ M=5\frac{{<V_{rad}}^2>r}{G}$$
Know more about the proof [here](https://en.wikipedia.org/wiki/Virial_theorem#Theoretical_analysis)


In [ ]:
v_rms=np.sqrt(np.power(new_data['Radial Velocity (km/s)'],2).mean())
velocity_dispersion=v_rms
distance_of_cluster=mean_dist
radius_of_cluster=distance_of_cluster * angular_size_of_cluster/2
constant=5
gravitational_constant=6.67
# 1parsec=3.086* 10^16 m  Msun=1.989 * 10^30
mass_of_galaxy=constant * ((velocity_dispersion)**2) *radius_of_cluster*3.086*1000/(gravitational_constant*1.989)#after taking out powers and unit conversions.
print("mass of cluster in terms of mass of sun")
print(mass_of_galaxy/1e+15,"*10^15 Msun",end="")


***

#### Q5.2 ESTIMATE the visible mass in the galaxy cluster. Use the Mass - Luminosity Relation for Main-Sequence Stars, assuming that it holds well for galaxies too. You may use the Milky Way as a reference.

Absolute Magnitude of Milky Way $ \approx -21 mag$

Visible mass of Milky Way $\approx 2 \times 10 ^ {11} M_{sun}$

#### Relation between abs_Mag and luminosity
Let $L_1$ ,$L_2$ and $M_1$,$M_2$ be the luminosity and Absolute Mag respectively

From the definiton of apparant mag ,$$ m1 -m2 =-2.5\log_{10}\frac{f1}{f2} \qquad \text{where f1 and f2 are fluxes}$$
since $f=\frac{L}{4\pi d^2}$, by taking d =1 parsec for both the stars, the equation becomes,
$$ M1 -M2 =-2.5\log_{10}\frac{L_1}{L_2} $$
since Absolute Mag is the apparent mag when the star is at 1 parsec, so we can replace m with M.


#### Relation between luminosity and Mass For main sequence stars 

$$\frac{L1}{L2}={\big(\frac{M1}{M2}\big)}^{3.5}$$
where $M_1$ and $M_2$ are masses of stars respectively

From the above two equations
$$\frac{Mass_1}{Mass_2}={100^{\big(\frac{{Mag}_2-{Mag}_1}{5}\big)}}^{\frac{1}{3.5}}$$

In [ ]:
# your code here
mag_milkyway=-21
new_data['mass']=2*np.power(np.power(100,(mag_milkyway-new_data['abs_MAG'])/5),1/3.5)#nelected the power of 10 in weight of milkyway 
total_mass=new_data['mass'].sum()
print("Total mass in units of 10 power 15:  ",total_mass*1e+11/(1e+15),"Msun","\n")#1e+11 power of 10 in weight of milky way galaxy
print("Percentage of weight of galaxies in the cluster:  ",total_mass*1e+11*100/mass_of_galaxy,"\n")#about 6 percent of the galaxy clusters weight is of stars 
percentage_of_dark_matter=100-(total_mass*1e+11*100/mass_of_galaxy)
print("Percentage of Dark matter:  ",percentage_of_dark_matter)

***

#### Q5.3 What is the fraction of the total mass of the galaxy cluster that is actually in the form of Dark Matter

weight of Dark matter constituets about 97.5% of the total mass of cluster.

***